In [23]:
# importing the python libraries
from IPython.display import display, Image, clear_output, HTML, Audio
from jupyter_ui_poll import ui_events
import ipywidgets as widgets
import time
import random
import os
import pandas as pd
from bs4 import BeautifulSoup

In [27]:
# Function to send data to Google Form
def send_to_google_form(data_dict, form_url):
    form_id = form_url.split('/d/')[1].split('/viewform')[0]
    view_form_url = f'https://docs.google.com/forms/d/e/1FAIpQLScLFJvQWuQ4voGLWNRkINu0vr1j35uWCZ3-zriKamc09tYpfg/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/1FAIpQLScLFJvQWuQ4voGLWNRkINu0vr1j35uWCZ3-zriKamc09tYpfg/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}

    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]

    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

# Function for waiting for an event
def wait_for_event(event_info, timeout=-1, interval=0.001, max_rate=20, allow_interrupt=True):    
    start_wait = time.time()
    n_proc = int(max_rate*interval)+1
    
    while True:
        # Process UI events

        # End loop if we have waited more than the timeout period
        if (timeout != -1) and (time.time() > start_wait + timeout):
            break
                
        # End loop if event has occurred
        if allow_interrupt and event_info['description'] != "":
            break
                
        time.sleep(interval)

# Function to register button event
def register_btn_event(btn, event_info):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()

# Callback function for submit button
def submit_button_callback(btn, age_dropdown, gender_radiobuttons):
    age = age_dropdown.value
    gender = gender_radiobuttons.value
    data = {'Age': age, 'Gender': gender}
    # Call send_to_google_form here with correct data and URL
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLScLFJvQWuQ4voGLWNRkINu0vr1j35uWCZ3-zriKamc09tYpfg/viewform'  # Replace with your Google Form URL
    send_to_google_form(data, form_url)
    clear_output(wait=False)

# Function to display the submit button
def submit_button(age_dropdown, gender_radiobuttons):
    confirm_btn = widgets.Button(description="Confirm")
    display(confirm_btn)
    event_info = {'type': "", 'description': "", 'time': -1}
    confirm_btn.on_click(lambda btn: submit_button_callback(btn, age_dropdown, gender_radiobuttons))
    wait_for_event()
    clear_output(wait=False)

# Main function to get personal information
def personal_info():
    print('Welcome to the Arithmetic test. This test will take approximately 3 minutes and the questions will progressively increase in difficulty.')
    print('Each part of the question will flash on your screen for 1.5 seconds after which you will be prompted to respond.')
    print('After completing the test, you will receive a score that accounts for the number of correct answers and the time taken to respond to each question.')
    time.sleep(4)
    clear_output(wait=False)

    print('How old are you?')
    age_dropdown = widgets.Dropdown(
        options=[('Select Age', None)] + [(str(age), age) for age in range(18, 30)],
        value=None
    )
    display(age_dropdown)

    print('What is your gender?')
    gender_radiobuttons = widgets.RadioButtons(
        options=['Male', 'Female', 'Other'],
        description='',
        disabled=False
    )
    display(gender_radiobuttons)

    submit_button(age_dropdown, gender_radiobuttons)

#fix this!!!!!

In [30]:
# Function to play a sound from a YouTube link
def play_sound(mp3_path):
    """
    Plays an MP3 sound.
    
    Parameters:
    mp3_path: str, the path to the MP3 file
    """
    display(Audio(mp3_path, autoplay=True))

def clear_screen():
    """
    Clears the output screen based on the operating system.
    """
    os.system('cls' if os.name == 'nt' else 'clear')

def flash_part(part, duration=2, mp3_path='/Users/gayatri/Desktop/introduction to coding for bioscience research/Courswork/beep.mp3'):
    """
    Displays each part of the question for a specified duration and then clears the screen.
    
    Parameters:
    part: str, the part of the question to display
    duration: int, the duration in seconds to display the part
    mp3_path: str, the path to the MP3 file
    """
    clear_screen()
    print(part, end='', flush=True)
    play_sound(mp3_path)  # Play sound
    time.sleep(duration)
    clear_output(wait=True)

# Function to handle the submission of an answer
def on_submit(btn):
    global correct, incorrect, times, current_index
    their_answer = answer_input.value
    time_to_answer = time.time() - start_time
    times.append(time_to_answer)
    answer_input.value = ''  # Clear the input field

    if int(their_answer) == answers[current_index]:
        print(f"{answers[current_index]} is correct! :)")
        correct += 1
    else:
        print(f"Incorrect :( The correct answer was: {answers[current_index]}")
        incorrect += 1

    print(f"Time taken for this question: {time_to_answer:.2f} seconds")
    
    if current_index < len(questions) - 1:
        current_index += 1
        time.sleep(2)
        clear_output(wait=True)
        display_question(current_index)  # Display next question
    else:
        calculate_score()

# Function to display the question using widgets
def display_question(index):
    """
    Displays each component of the question.
    
    Parameters:
    index: int, the index of the current question
    """
    global start_time
    question = questions[index]
    
    for part in question.split():
        style = "color: aqua; font-size: 100px; display: flex; justify-content: center; align-items: center;"
        display(HTML(f"<div style='{style}'><span>{part}</span></div>"))
        flash_part(part)
        clear_output(wait=True)

    start_time = time.time()
    display(answer_input)
    answer_input.on_submit(on_submit)

# Function to calculate and display the final score
def calculate_score():
    """
    Calculates the final score based on the number of correct and incorrect answers and the time taken.
    """
    total_time = (len(questions)*6)-sum(times) #multiply by 6 as each question has three parts that are flashed for two seconds each.
    print(f"Test completed in {total_time:.2f} seconds.")
    score_percentage = (correct / (correct + incorrect)) * 100
    print(f"You got: {correct} questions correct out of {len(questions)} ({score_percentage:.2f}%)")

    hypothesis_time = hypothesis_time_per_question * len(questions)
    score_time = (correct - ((total_time - hypothesis_time) * 0.25) / ((correct + incorrect))*100)
    print(f"Your final score is {score_time:.2f}%.")

# Main test function
def run_math():
    personal_info()
    wait_for_event(e)
    """
    Runs the math test by displaying each question and collecting answers.
    """
    display_question(0)  # Display first question

# Global variables
correct = 0
incorrect = 0
times = []
current_index = 0
hypothesis_time_per_question = 4
start_time = 0

# Widget for the answer input
answer_input = widgets.Text(
    placeholder='Type your answer here...',
    description='Answer:',
    disabled=False
)

# Questions and answers
questions = ['2 + 2', '3 + 6', '9 + 7', '18 - 17', '20 - 27']
answers = [4, 9, 16, 1, -7]

# Run the test
run_math()


How old are you?


Dropdown(options=(('Select Age', None), ('18', 18), ('19', 19), ('20', 20), ('21', 21), ('22', 22), ('23', 23)…

What is your gender?


RadioButtons(options=('Male', 'Female', 'Other'), value='Male')

Button(description='Confirm', style=ButtonStyle())

KeyboardInterrupt: 

In [ ]:
# write a code for a start button the begin the test
# format the questions in a pretty way
# finish creating all the questions
# link to a google form
# add a way of allowing users to view their performance in comparison to the class dataset results
# make interface more pretty



In [50]:
run_math(questions, answers)


4 is correct! :)
Time taken for this question: 1.24 seconds
Test completed in 5.52 seconds.
You got: 1 questions correct out of 1 (100.00%)
Your final score is 86.93%.
